In [1]:
import collections
%matplotlib inline
import matplotlib
import numpy as np
import openml
import pandas as pd
import time
print(openml.config.server)

https://www.openml.org/api/v1/xml


In [2]:
automlbench_dids = [41021, 42225, 42571, 4549, 42572, 42570, 42705,
                   42728, 550, 546, 541, 507, 505, 287, 216, 41540,
                   42688, 422, 416, 42724, 42727, 42729, 42726, 42730,
                   201, 41980, 42731, 531, 42563, 574, 3050, 3277, 43071]

In [3]:
all_datasets = openml.datasets.list_datasets()
to_consider = openml.datasets.list_datasets()
TOTAL = len(all_datasets)
# print(TOTAL)
# # all_tasks = openml.tasks.list_tasks()
to_remove = set()
# # print(len(all_tasks))
# print("From:", len(to_consider)) # 3004

In [4]:
# remove datasets from the datasets list which are in the automl bench or close relatives
print("From:", len(to_consider))

# remove exact duplicates
for did in all_datasets:
    if did in automlbench_dids:
        to_remove.add(did)

for did in to_remove:
    if did in to_consider:
        del to_consider[did]
assert len(to_consider) == len(all_datasets) - len(automlbench_dids)

print("To:", len(to_consider)) # 2965

From: 5180
To: 5147


In [5]:
# Remove datasets with too few features, instances or classes
print("From:", len(to_consider))

for did in all_datasets:
    try:
        if all_datasets[did]['NumberOfInstances'] * all_datasets[did]['NumberOfFeatures'] < 2_000:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfInstances'] >= 500_000:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfInstances'] < 20:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfFeatures'] < 2:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfFeatures'] > 2500:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfClasses'] > 1:
            to_remove.add(did)
        elif all_datasets[did]['NumberOfInstances'] * all_datasets[did]['NumberOfFeatures'] > 10_000_000:
            to_remove.add(did)
    except KeyError:
        #print(did, all_datasets[did])
        to_remove.add(did)
        
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
        
print("To:", len(to_consider)) # 651

From: 5147
To: 1510


In [6]:
all_datasets_as_frame = pd.DataFrame.from_dict(all_datasets, orient='index')

In [7]:
# Remove all datasets with "BNG" in name
print("From:", len(to_consider))

for did in to_consider:
    if to_consider[did]['name'].startswith("BNG"):
        to_remove.add(did)
        
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
        
print("To:", len(to_consider)) # 641

From: 1510
To: 1503


In [8]:
# check for similar names and manually check overlaps
# def return_triplets(name):
#     triplets = set()
#     name = name.lower()
#     for i in range(len(name) - 2):
#         triplets.add(name[i: i+3])
#     return triplets

# manually_look_at = []
# for did in to_consider:
#     for did2 in automlbench_dids:
#         automl_triplets = return_triplets(all_datasets[did2]['name'])
#         other_triplets = return_triplets(all_datasets[did]['name'])
#         n_hits = sum([trip in other_triplets for trip in automl_triplets])
#         ratio = n_hits / len(automl_triplets)
#         if n_hits > 2:
#             manually_look_at.append([n_hits, ratio, all_datasets[did]['name'], all_datasets[did2]['name'], did, did2])

# df = pd.DataFrame(manually_look_at)
# with open("Manualautomlregr.csv", "w") as fh:
#    fh.write(df.to_csv())     


In [9]:
# This list contains all manually collected dataset ids as in 
# https://docs.google.com/spreadsheets/d/1XH1mQFq7-1fs28F-CQJo6i6Wg7IlpDS0aDTAJBoaD7k/edit#gid=0
print("From:", len(to_consider))
for did in [209, 543, 4545, 42092, 42635, 42723, 43465, 43984, 43986, 
            43989, 43992, 43994, 43996, 43997, 43998, 43999, 44002, 
            44009, 44011, 44013, 44014, 44015, 44016, 44019, 44039, 
            44042, 44044, 44047, 44049, 44051, 44054, 44057, 44059, 
            44061, 44062, 44063, 44064, 44066, 44134, 44136, 44138, 
            44139, 44140, 4414, 44142, 44144, 44152, 44201, 44202, 
            44203, 44269, 44956, 44979, 44985, 44987, 44988, 44990, 
            45031, 45032, 45033, 45041, 45042, 42713, 44000, 44017, 44048, 44063, 44142]:
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 603

From: 1503
To: 1438


In [10]:
# # check for similar metafeatures and manually check overlaps
# metafeature_names = [
#     'MaxNominalAttDistinctValues', 'NumberOfClasses',
#     'NumberOfFeatures', 'NumberOfInstances', 'NumberOfInstancesWithMissingValues', 'NumberOfMissingValues',
#     'NumberOfNumericFeatures', 'NumberOfSymbolicFeatures',
# ]

# simple_metafeatures_to_consider = pd.DataFrame(
#     {k: v for k, v in all_datasets.items() if k in to_consider}
# ).transpose()[metafeature_names]
# simple_metafeatures_automl = pd.DataFrame(
#     {k: v for k, v in all_datasets.items() if k in automlbench_dids}
# ).transpose()[metafeature_names]

# # Checking the hamming distance of the datasets
# manually_look_at = []
# for did1, mf1 in simple_metafeatures_to_consider.iterrows():
#     for did2, mf2 in simple_metafeatures_automl.iterrows():
#         n_hits = np.sum(mf1 == mf2)
#         ratio = n_hits / len(mf1)
#         n_hits = np.sum(
#             mf1[['NumberOfFeatures', 'NumberOfInstances']] == mf2[['NumberOfFeatures', 'NumberOfInstances']]
#         )
#         if ratio > 0.5 or n_hits == 2:
#             entry = [ratio, n_hits, all_datasets[did1]['name'], all_datasets[did2]['name'], did1, did2]
#             mfs = np.array([[mf1[n], mf2[n]] for n in metafeature_names]).flatten()
#             entry.extend(mfs)
#             manually_look_at.append(entry)

# header = ["ratio", "n_hits", "name", "automl name", "did", "automl did"] 
# mfs = list(np.array([[m, m] for m in metafeature_names]).flatten())
# df = pd.DataFrame(manually_look_at, columns = header + mfs)
# # with open("ManualMetafeaturesregr.csv", "w") as fh:
# #    fh.write(df.to_csv())            

In [11]:
# This list contains all manually collected dataset ids as in 
# https://docs.google.com/spreadsheets/d/1SWEBsFTykdUMq-wBcx9NHchhdolB632EJYaAmQwwQtM/edit#gid=0
print("From:", len(to_consider))
for did in [315, 580, 610, 652, 41938, 41968, 41969, 43070, 43672, 43888, 43983, 43991,
            44008, 44133, 44141, 44971, 44972, 44989, 45030]:
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 599
# check whether we can work usefully with creditcard (1597)

From: 1438
To: 1419


In [12]:
pd.set_option('display.max_rows', 25)
columns = ['name', 'NumberOfClasses', 'NumberOfFeatures', 'NumberOfInstances', ]

to_consider_stats = pd.DataFrame(
    {k: v for k, v in all_datasets.items() if k in to_consider}
).transpose()[columns]

to_consider_stats

,name,NumberOfClasses,NumberOfFeatures,NumberOfInstances
8,liver-disorders,0.0,6.0,345.0
189,kin8nm,0.0,9.0,8192.0
191,wisconsin,0.0,33.0,194.0
194,cleveland,0.0,14.0,303.0
195,auto_price,0.0,16.0,159.0
...,...,...,...,...
45076,churn,0.0,21.0,5000.0
45078,nomao,0.0,119.0,34465.0
45079,road-safety,0.0,33.0,111762.0
45081,Tallo,0.0,21.0,307014.0


In [13]:
# # remove sparse datasets
print("From:", len(to_consider))
start = time.time()
for i, did in enumerate(to_consider):
    #print(did)
    if (i+1) % 25 == 0:
        print("%4.2fsec: %d/%d" % (time.time() - start, i+1, len(to_consider)))
    d = openml.datasets.get_dataset(int(did), download_data=False)
    #print(
    #    d.format.lower(), 
    #    'sparse' in d.format.lower(), 
    #    d.description and 'CLASSINDEX: none specific' in d.description
    #)
    # if 'sparse' in d.format.lower():
    #     to_remove.add(did)
    # if d.description and 'CLASSINDEX: none specific' in d.description:
    #     to_remove.add(did)
    for f in d.features.values():
        if f.data_type == 'string':
            to_remove.add(did)

for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 599

From: 1419
0.06sec: 25/1419
0.11sec: 50/1419
0.20sec: 75/1419
0.47sec: 100/1419
0.65sec: 125/1419
0.82sec: 150/1419
0.93sec: 175/1419
1.18sec: 200/1419
1.60sec: 225/1419
1.94sec: 250/1419
2.32sec: 275/1419
2.75sec: 300/1419
3.17sec: 325/1419
3.55sec: 350/1419
3.87sec: 375/1419
4.33sec: 400/1419
4.70sec: 425/1419
5.05sec: 450/1419
5.44sec: 475/1419
5.97sec: 500/1419
6.40sec: 525/1419
6.90sec: 550/1419
7.53sec: 575/1419
8.17sec: 600/1419
8.76sec: 625/1419
9.15sec: 650/1419
9.70sec: 675/1419
10.21sec: 700/1419
10.70sec: 725/1419
11.20sec: 750/1419
11.61sec: 775/1419
12.07sec: 800/1419
12.49sec: 825/1419
12.99sec: 850/1419
13.48sec: 875/1419
13.97sec: 900/1419
14.48sec: 925/1419
14.89sec: 950/1419
15.42sec: 975/1419
15.99sec: 1000/1419
16.48sec: 1025/1419
16.95sec: 1050/1419
17.35sec: 1075/1419
17.76sec: 1100/1419
18.20sec: 1125/1419
18.58sec: 1150/1419
18.84sec: 1175/1419
18.97sec: 1200/1419
19.11sec: 1225/1419
19.32sec: 1250/1419
19.44sec: 1275/1419
19.65sec: 1300/1419
20.03sec: 1325/141

In [14]:
# # remove identical/similar datasets (to reduce the load computing the meta-data)
# # check for similar names and manually check overlaps
# def return_triplets(name):
#     triplets = set()
#     name = name.lower()
#     for i in range(len(name) - 2):
#         triplets.add(name[i: i+3])
#     return triplets

# #Remove QSAR datasets

# manually_look_at = []
# for i, did in enumerate(to_consider):
#     for j, did2 in enumerate(to_consider):
        
#         if all_datasets[did]["name"].startswith("QSAR") or all_datasets[did2]["name"].startswith("QSAR"):
#             continue
#         if j <= i:
#             continue
#         if len(all_datasets[did2]['name']) < 3:
#             continue
#         other2_triplets = return_triplets(all_datasets[did2]['name'])
#         other_triplets = return_triplets(all_datasets[did]['name'])
#         n_hits = sum([trip in other_triplets for trip in other2_triplets])
#         ratio = n_hits / len(other2_triplets)
#         if n_hits > 2:
#             manually_look_at.append([n_hits, ratio, all_datasets[did]['name'], all_datasets[did2]['name'], did, did2])

# df = pd.DataFrame(manually_look_at)
# with open("manual_check.csv", "w") as fh:
#    fh.write(df.to_csv())   

In [15]:
print("From:", len(to_consider))
for did in [45013, 45014, 44980, 195, 42372, 573, 43981, 43982, 43990,
            44007, 44132, 45029, 44978, 43882, 42900, 562, 43987, 43995, 44012, 44137,
            555, 556, 44055, 43127, 504, 568, 570, 575, 576, 577, 578, 670, 671, 672,
            44041, 44056, 43056, 1436, 1435, 44965, 44003, 44020, 23515,42436, 42437, 
            42438, 42439, 42444, 42445, 41514, 41515, 41516, 41517, 41518, 41519, 43384, 
            43483, 43582, 43943, 43959, 44187, 44191, 44192, 44194, 44223, 41516, 41517, 43123, 
            41523, 41524, 41525, 43127, 44236, 41928, 42131, 42559, 42720, 43928, 44004, 44021, 
            44146, 45048, 42559, 42720, 43881, 43882, 44235, 43127, 44068, 44069, 43144, 44069, 43871, 44967,
            42352, 44029, 44069, 44962, 43440, 42437, 42438, 42439, 42444, 42445, 45043, 42822, 43682, 43050,
            43056, 44957, 43079, 44055, 44005, 44022, 44147, 44983, 44006, 44023, 44148, 44964, 43180, 43190, 
            43192, 43194, 43198, 43256, 44179, 44180, 44214, 44218, 44219, 44220, 44221, 44267, 43483, 43582,
            43874, 43403, 43665, 43759, 43466, 43692, 44976, 43962, 44960, 44025, 43939, 43979, 44024, 44031, 
            44090, 44362, 44363, 44364, 44365, 44366, 44977, 45025, 44053, 44252, 44280, 44243, 44252, 44280, 
            44794, 44296, 44310, 45015] :
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 503

From: 1344
To: 1238


In [16]:
# Subsample from 'topics', where there are dozens of super-similar
# datasets from one source
dataset_names = {did: value['name'] for did, value in all_datasets.items()
                 if did in to_consider}

rng = np.random.RandomState(1)

friedman_datasets = [
    did for did, name in dataset_names.items() if name.startswith('fri_c')
]
print(friedman_datasets)

qsar_datasets =  [
    did for did, name in dataset_names.items() 
    if name.startswith('QSAR')
]
print(qsar_datasets)

# print(qsar_datasets)
for dataset_ids_to_sample_from, num_keep in (
    (friedman_datasets, 10),
    # (ova_datasets, 7),
    # (volcanoes_datasets, 10),
    # (auto_univ_datasets, 4),
    (qsar_datasets, 50)
):
    choices_to_drop = np.random.choice(
        dataset_ids_to_sample_from,
        replace=False,
        size=len(dataset_ids_to_sample_from) - num_keep)
    for choice in choices_to_drop:
        if choice in to_consider:
            del to_consider[choice]
print("To:", len(to_consider)) # 454

[581, 582, 583, 584, 586, 587, 588, 589, 590, 592, 593, 595, 597, 598, 599, 600, 602, 603, 604, 605, 606, 607, 608, 609, 612, 614, 615, 616, 617, 618, 619, 620, 622, 623, 625, 626, 627, 628, 629, 630, 631, 632, 633, 635, 636, 637, 638, 639, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 653, 654, 655, 657, 658]
[3040, 3042, 3043, 3044, 3045, 3046, 3047, 3048, 3049, 3051, 3052, 3053, 3054, 3056, 3058, 3059, 3060, 3061, 3062, 3063, 3064, 3065, 3066, 3067, 3068, 3069, 3070, 3071, 3072, 3073, 3074, 3075, 3076, 3077, 3078, 3079, 3080, 3081, 3082, 3084, 3085, 3086, 3088, 3089, 3090, 3091, 3092, 3093, 3094, 3096, 3097, 3098, 3100, 3101, 3103, 3104, 3105, 3106, 3107, 3110, 3111, 3112, 3113, 3114, 3115, 3116, 3117, 3118, 3119, 3120, 3121, 3122, 3123, 3124, 3125, 3126, 3127, 3128, 3129, 3130, 3131, 3132, 3133, 3134, 3135, 3136, 3138, 3139, 3140, 3141, 3142, 3143, 3144, 3145, 3146, 3147, 3149, 3150, 3151, 3152, 3153, 3154, 3155, 3156, 3157, 3158, 3159, 3161, 3162, 3163, 3164, 3165, 3166, 

In [17]:
# # check for similar metafeatures and manually check overlaps
# metafeature_names = [
#     'MajorityClassSize', 'MaxNominalAttDistinctValues', 'MinorityClassSize', 'NumberOfClasses',
#     'NumberOfFeatures', 'NumberOfInstances', 'NumberOfInstancesWithMissingValues', 'NumberOfMissingValues',
#     'NumberOfNumericFeatures', 'NumberOfSymbolicFeatures',
# ]

# simple_metafeatures_to_consider = pd.DataFrame(
#     {k: v for k, v in all_datasets.items() if k in to_consider}
# ).transpose()[metafeature_names]

# # Checking the hamming distance of the datasets
# manually_look_at = []
# for did1, mf1 in simple_metafeatures_to_consider.iterrows():
#     for did2, mf2 in simple_metafeatures_to_consider.iterrows():
#         if did2 <= did1:
#             continue
#         n_hits = np.sum(mf1 == mf2)
#         ratio = n_hits / len(mf1)
#         n_hits = np.sum(
#             mf1[['NumberOfFeatures', 'NumberOfInstances']] == mf2[['NumberOfFeatures', 'NumberOfInstances']]
#         )
#         if ratio > 0.5 or n_hits == 2:
#             entry = [ratio, n_hits, all_datasets[did1]['name'], all_datasets[did2]['name'], did1, did2]
#             mfs = np.array([[mf1[n], mf2[n]] for n in metafeature_names]).flatten()
#             entry.extend(mfs)
#             manually_look_at.append(entry)

# header = ["ratio", "n_hits", "name", "automl name", "did", "automl did"] 
# mfs = list(np.array([[m, m] for m in metafeature_names]).flatten())
# df = pd.DataFrame(manually_look_at, columns = header + mfs)
# with open("ManualMetafeaturesregression.csv", "w") as fh:
#    fh.write(df.to_csv()) 

In [18]:
print("From:", len(to_consider))
for did in [44026, 45055, 45052, 45053, 45054, 45055, 1027, 43988, 42111, 42112, 42113, 42361,
            1424, 1425, 1426, 1427, 1428, 1429,]:
    to_remove.add(did)
 
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 222

From: 258
To: 242


In [19]:
print("From:", len(to_consider))

for did in [
     1414, #bad attribute error when retrieving dataset
     43026, #single entry dataset
     43034, #single entry dataset
     1414, #bucket not accessible
     41275, #bucket not accessible
     45108, #bucket not accessible
     41275, #bucket not accessible
     44537, #more classes than instances
     42130, #errors in gama
     5, #more classes than instances
     41700, #timeseries
     42803, #too big
     231, #below are all classification tasks with feature NumberofClasses 0
     298,
     301,
     516,
     524,
     703,
     1028,
     1097,
     1228,
     1430,
     1432,
     1433,
     1571,
     1572,
     1574,
     1575,
     1579,
     1589,
     1591,
     1593,
     4532,
     23395,
     41463,
     41943,
     42175,
     42176,
     42464,
     42636,
     42889,
     42931
]:
    to_remove.add(did)
    
for did in to_remove:
    if did in to_consider:
        del to_consider[did]
print("To:", len(to_consider)) # 209

From: 242
To: 214


In [32]:
final = pd.DataFrame(
    {k: v for k, v in all_datasets.items() if k in to_consider}
).transpose()
final.drop_duplicates(subset = ["name"], keep = "last", inplace = True)


In [34]:
pd.set_option('display.max_rows', None)

In [35]:
final

,did,name,version,uploader,status,format,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures,MaxNominalAttDistinctValues
8,8,liver-disorders,1,1,active,ARFF,0.0,6.0,345.0,0.0,0.0,6.0,0.0,NaN
189,189,kin8nm,1,1,active,ARFF,0.0,9.0,8192.0,0.0,0.0,9.0,0.0,NaN
191,191,wisconsin,1,1,active,ARFF,0.0,33.0,194.0,0.0,0.0,33.0,0.0,NaN
194,194,cleveland,1,1,active,ARFF,0.0,14.0,303.0,6.0,6.0,7.0,7.0,4.0
196,196,autoMpg,1,1,active,ARFF,0.0,8.0,398.0,6.0,6.0,5.0,3.0,13.0
197,197,cpu_act,1,1,active,ARFF,0.0,22.0,8192.0,0.0,0.0,22.0,0.0,NaN
198,198,delta_elevators,1,1,active,ARFF,0.0,7.0,9517.0,0.0,0.0,7.0,0.0,NaN
200,200,pbc,1,1,active,ARFF,0.0,19.0,418.0,142.0,1239.0,11.0,8.0,4.0
204,204,cholesterol,1,1,active,ARFF,0.0,14.0,303.0,6.0,6.0,7.0,7.0,4.0
206,206,triazines,1,1,active,ARFF,0.0,61.0,186.0,0.0,0.0,61.0,0.0,NaN
